<a href="https://colab.research.google.com/github/laurahallaman1/final_project/blob/master/Code/2predictReviewer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [0]:
 # Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NaiveBayes").getOrCreate()

In [3]:

from pyspark import SparkFiles
url ="https://raw.githubusercontent.com/laurahallaman1/final_project/master/Book3.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Book3.csv"), sep=",", header=True)

# Show DataFrame
df.show(30)

+--------------------+--------------------+--------------------+--------------------+-----------+----------+------+--------------------+--------------------+---------------+--------+
|                  id|                name|               asins|   primaryCategories|doRecommend|numHelpful|rating|                text|               title|       username|reviewer|
+--------------------+--------------------+--------------------+--------------------+-----------+----------+------+--------------------+--------------------+---------------+--------+
|AVpgNzjwLJeJML43Kpxn|AmazonBasics AAA ...|B00QWO9P0O,B00LH3...|     Health & Beauty|       null|      null|     4|Seems to work jus...|                Good|     Bycherubs1|  Person|
|AVqVGZNvQMlgsOJE6eUY|"All-New Kindle E...|          B00ZV9PXP2|         Electronics|      FALSE|         0|     3|This was advertis...|Paper white is th...|       Kindler1|  Person|
|AVqVGWLKnnc1JgDc3jF1|Fire Kids Edition...|          B018Y23MNM|         Electronics|

In [4]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df = df.withColumn('length', length(df['text']))
data_df.show()

+--------------------+--------------------+--------------------+--------------------+-----------+----------+------+--------------------+--------------------+---------------+--------+------+
|                  id|                name|               asins|   primaryCategories|doRecommend|numHelpful|rating|                text|               title|       username|reviewer|length|
+--------------------+--------------------+--------------------+--------------------+-----------+----------+------+--------------------+--------------------+---------------+--------+------+
|AVpgNzjwLJeJML43Kpxn|AmazonBasics AAA ...|B00QWO9P0O,B00LH3...|     Health & Beauty|       null|      null|     4|Seems to work jus...|                Good|     Bycherubs1|  Person|    85|
|AVqVGZNvQMlgsOJE6eUY|"All-New Kindle E...|          B00ZV9PXP2|         Electronics|      FALSE|         0|     3|This was advertis...|Paper white is th...|       Kindler1|  Person|   316|
|AVqVGWLKnnc1JgDc3jF1|Fire Kids Edition...|       

In [0]:
 from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
reviewer_to_num = StringIndexer(inputCol='reviewer',outputCol='label')
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="token_text", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [0]:
 from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [0]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[reviewer_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [0]:
 # Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(data_df)
cleaned = cleaner.transform(data_df)

In [9]:
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|(262145,[16332,20...|
|  1.0|(262145,[5795,163...|
|  0.0|(262145,[16332,84...|
|  2.0|(262145,[6258,706...|
|  0.0|(262145,[35263,37...|
|  1.0|(262145,[3924,963...|
|  1.0|(262145,[12888,14...|
|  0.0|(262145,[4869,963...|
|  1.0|(262145,[9639,244...|
|  1.0|(262145,[2306,963...|
|  1.0|(262145,[7367,249...|
|  1.0|(262145,[18691,37...|
|  0.0|(262145,[68056,91...|
|  1.0|(262145,[9639,163...|
|  1.0|(262145,[83936,11...|
|  0.0|(262145,[9639,163...|
|  1.0|(262145,[14,254,4...|
|  0.0|(262145,[9639,473...|
|  1.0|(262145,[7062,963...|
|  0.0|(262145,[16426,25...|
+-----+--------------------+
only showing top 20 rows



In [0]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [11]:
test_results = predictor.transform(testing)
test_results.show(30)

+--------------------+--------------------+--------------------+-----------------+-----------+----------+------+--------------------+--------------------+--------------------+---------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|                  id|                name|               asins|primaryCategories|doRecommend|numHelpful|rating|                text|               title|            username| reviewer|length|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+-----------------+-----------+----------+------+--------------------+--------------------+--------------------+---------+------+-----+--------------------+--------------------+--------------------+--------------------+----

In [12]:
 # Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.438471


In [17]:
test_results.select(['title', 'stop_tokens','text','reviewer']).show()

+--------------------+--------------------+--------------------+---------+
|               title|         stop_tokens|                text| reviewer|
+--------------------+--------------------+--------------------+---------+
|Why yet another c...|[reason., product...|No reason. This p...|   Person|
|     Nope, not again|[3/4, package, du...|3/4 of the packag...|      Bot|
|dont waiste your ...|[bought, a,, c, d...|Bought a, c d 9v ...|  userBot|
|They WILL burn up...|[burnt, copper, s...|Burnt up a copper...|   Person|
|        Short life!!|[bought, several,...|I bought these se...|  userBot|
|I must have recei...|[high, hopes, sur...|I had high hopes ...|  userBot|
|           Two Stars|[bad, shelf, life...|Bad shelf life. N...|      Bot|
|           Two Stars|  [go, brand, name.]|Go with a brand n...|      Bot|
|Very disappointed...|[disappointed, ba...|Very disappointed...|  userBot|
|Great for some ap...|[great, value, ki...|Great value for k...|  userBot|
|         Three Stars|   